In [1]:
import pyspark

In [32]:
import pandas as pd
import numpy as np
from pydataset import data
from pyspark.sql.functions import col, expr, lit, regexp_extract, regexp_replace, avg

In [3]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/05 12:11:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [4]:
languages = {"language":["Python", "Java", "Ruby", "JavaScript"]}

In [5]:
df = spark.createDataFrame(pd.DataFrame(languages))

In [6]:
def pyspark_shape(df):
    print(f"{df.count()}, {len(df.columns)}")

In [7]:
pyspark_shape(df)

4, 1


In [8]:
df.show(7)

+----------+
|  language|
+----------+
|    Python|
|      Java|
|      Ruby|
|JavaScript|
+----------+



# 2.Load the mpg dataset as a spark dataframe.

- Create 1 column of output that contains a message like the one below:


`The 1999 audi a4 has a 4 cylinder engine.`
For each vehicle.

In [9]:
mpg = spark.createDataFrame(data("mpg"))

In [10]:
mpg.first()['year'], mpg.first()['manufacturer'], mpg.first()['model'], mpg.first()['cyl']

(1999, 'audi', 'a4', 4)

- Transform the trans column so that it only contains either manual or auto.

In [11]:
mpg.filter((mpg.trans.startswith('m'))|(mpg.trans.startswith('a'))).show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



# 3. Load the tips dataset as a spark dataframe.

In [12]:
tips = spark.createDataFrame(data("tips"))

In [13]:
pyspark_shape(tips)

244, 7


- What percentage of observations are smokers?

In [14]:
round(tips.filter(tips.smoker.startswith("Y")).count()/tips.count(),3)

0.381

- Create a column that contains the tip percentage

In [16]:
tips = tips.withColumn("tip_percentage", tips.tip/tips.total_bill)

In [18]:
tips.show(5)

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentage|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
+----------+----+------+------+---+------+----+-------------------+
only showing top 5 rows



- Calculate the average tip percentage for each combination of sex and smoker.

In [33]:
tips.groupBy("smoker").agg(avg("tip_percentage")).collect()

[Row(smoker='No', avg(tip_percentage)=0.15932846217921526),
 Row(smoker='Yes', avg(tip_percentage)=0.16319604463687792)]

In [34]:
tips.groupBy("sex").agg(avg("tip_percentage")).collect()

[Row(sex='Female', avg(tip_percentage)=0.16649073632892478),
 Row(sex='Male', avg(tip_percentage)=0.15765054700429743)]

# 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).